In [ ]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False
import numpy as np
import os

### Functions

In [ ]:
def create_coordinates(radius=100.0, bdsNumber=1000):
    """3D random walk generator"""
    r = 1.0  # initial bond length
    x = [np.random.uniform(-3, 3)]
    y = [np.random.uniform(-3, 3)]
    z = [np.random.uniform(-3, 3)]
    for i in range(bdsNumber - 1):
        theta = np.arccos(1 - 2 * np.random.uniform(0, 1))
        phi = np.random.uniform(0, 2 * np.pi)
        while (
            np.sqrt(
                (x[-1] + r * np.sin(theta) * np.cos(phi)) ** 2
                + (y[-1] + r * np.sin(theta) * np.sin(phi)) ** 2
                + (z[-1] + r * np.cos(theta)) ** 2
            )
            > radius
        ):
            theta = np.arccos(1 - 2 * np.random.uniform(0, 1))
            phi = np.random.uniform(0, 2 * np.pi)
        x.append(x[-1] + r * np.sin(theta) * np.cos(phi))
        y.append(y[-1] + r * np.sin(theta) * np.sin(phi))
        z.append(z[-1] + r * np.cos(theta))
    return np.array(x) + radius, np.array(y) + radius, np.array(z) + radius


def write_lmpdat_stat(chain_length=400, path="./chain400", box_size=45.42, n_chains=1):
    """Write single polymer chain of 400 beads.
    100 beads from both ends are just to avoid artefacts at boundaries.
    Then 200 beads to perform analysis."""

    with open(path, "w") as f:
        f.write(f"#linear chain of {chain_length*n_chains} beads\n\n")
        f.write(f"    {chain_length*n_chains} atoms\n")
        f.write(f"    {(chain_length*n_chains - n_chains)} bonds\n\n\n")
        f.write("  3 atom types\n")
        f.write("  2 bond types\n\n")
        f.write(f"0.000 {box_size:5.3f} xlo xhi\n")
        f.write(f"0.000 {box_size:5.3f} ylo yhi\n")
        f.write(f"0.000 {box_size:5.3f} zlo zhi\n\n")
        f.write("Masses\n\n1\t1000\n2\t1000\n3\t1000\n\n Atoms\n\n")

        for n_ch in range(n_chains):
            x, y, z = create_coordinates(radius=box_size / 2, bdsNumber=chain_length)
            for i in range(chain_length):
                f.write(
                    f"{n_ch*chain_length+i+1:8d} 0 1 {x[i]:5.4f} {y[i]:5.4f} {z[i]:5.4f}\n"
                )
        f.write("\n Bonds\n\n")
        for n_ch in range(n_chains):
            for i in range(1 + n_ch * chain_length, n_ch * chain_length + chain_length):
                f.write(f"{i:8d} 1 {i:5d} {i+1:5d}\n")

### Generate initial files

In [ ]:
chain_len = 1000  # beads
chromatin_conc = 0.1  # 10 %
cubic_box_size = (chain_len * 4 / 3 * np.pi / chromatin_conc) ** (1 / 3)

for t in range(9):
    for i in range(102):
        folder = f"path/to/system_set_{t}/replicate_{i}/"
        os.makedirs(folder, exist_ok=True)
        write_lmpdat_stat(
            chain_length=chain_len,
            path=f"{folder}/chain{chain_len}.dat",
            box_size=cubic_box_size,
            n_chains=1,
        )
    print(f"{t} done")